In [62]:
import os, sys
sys.path.append(os.path.join(os.path.dirname("/home/wsr217/mpararel/")))

In [63]:
from tqdm import tqdm
from dataset.translate_templates import *
import numpy as np

In [64]:
translator = TRANSLATOR_TO_OBJECT[Translator.GOOGLE]

In [65]:
K_POPULATED_TEMPLATES = 10

In [66]:
templates_folder = '/home/wsr217/mpararel/data/pararel'

In [67]:
tuples_folder = '/home/wsr217/mpararel/generated_datasets/mpararel_bing'

In [68]:
for relation_filename in tqdm(os.listdir(templates_folder)):
    templates = get_templates(os.path.join(templates_folder, relation_filename))
    if relation_filename == "P36.jsonl":
        break

 82%|████████▏ | 32/39 [00:00<00:00, 809.80it/s]


In [69]:
templates

[{'pattern': 'The capital of [X] is [Y] .',
  'lemma': 'capital',
  'extended_lemma': 'capital-of',
  'tense': 'present'},
 {'pattern': 'The capital city of [X] is [Y].',
  'lemma': 'capital-city',
  'extended_lemma': 'capital-city-of',
  'tense': 'present'},
 {'pattern': '[Y] is the capital of [X].',
  'lemma': 'capital',
  'extended_lemma': 'capital-of',
  'tense': 'present'},
 {'pattern': '[Y] is the capital city of [X].',
  'lemma': 'capital-city',
  'extended_lemma': 'capital-city-of',
  'tense': 'present'},
 {'pattern': "[X]'s capital, [Y].",
  'lemma': 'capital',
  'extended_lemma': 'capital-of',
  'tense': 'present'},
 {'pattern': "[X]'s capital city, [Y].",
  'lemma': 'capital-city',
  'extended_lemma': 'capital-city-of',
  'tense': 'present'},
 {'pattern': "[X]'s capital is [Y].",
  'lemma': 'capital',
  'extended_lemma': 'capital-of',
  'tense': 'present'},
 {'pattern': "[X]'s capital city is [Y].",
  'lemma': 'capital-city',
  'extended_lemma': 'capital-city-of',
  'tense':

In [70]:
def get_k_subject_object_tuples(filename, k_tuples):
    tuples = {}
    added = set()
    with open(filename) as fp:
        for i, line in enumerate(fp):
            line_data = json.loads(line)
            if (line_data["sub_label"] not in added and
                    line_data["obj_label"] not in added):
                tuples[line_data["lineid"]] = (line_data["sub_label"], line_data["obj_label"])
                added.add(line_data["sub_label"])
                added.add(line_data["obj_label"])
            if len(tuples) == k_tuples:
                break
    return tuples

In [71]:
def get_subject_object_tuples_from_lines(filename, lineas_to_add):
    tuples = {}
    with open(filename) as fp:
        for i, line in enumerate(fp):
            line_data = json.loads(line)
            if line_data["lineid"] in lineas_to_add:
                tuples[line_data["lineid"]] = (line_data["sub_label"], line_data["obj_label"])
    return tuples

In [72]:
def get_populated_phrases(template, tuples):
    populated_phrases = []
    for obj, sub in tuples:
        phrase = template.replace("[X]", obj)
        populated_phrases.append(phrase.replace("[Y]", sub))
    return populated_phrases

In [73]:
def longest_common_subsequence(string_list_1, string_list_2):
    m = len(string_list_1)
    n = len(string_list_2)
    # L[i][j] contains the length of the LCS of X[0..i-1] and Y[0..j-1].
    L = [[None]*(n + 1) for i in range(m + 1)]
    sequence = [[None]*(n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
                sequence[i][j] = ''
            elif string_list_1[i-1] == string_list_2[j-1]:
                L[i][j] = L[i-1][j-1] + 1
                sequence[i][j] = sequence[i-1][j-1] + ' ' + string_list_1[i-1]
            else:
                if L[i-1][j] > L[i][j-1]:
                    L[i][j] = L[i-1][j]
                    sequence[i][j] = sequence[i-1][j]
                else:
                    L[i][j] = L[i][j-1]
                    sequence[i][j] = sequence[i][j-1]
    return '' if not sequence[m][n] else sequence[m][n][1:]

In [84]:
def longest_common_subsequence(string_list_1, string_list_2):
    print("searching lcs between:",string_list_1, string_list_2)
    m = len(string_list_1)
    n = len(string_list_2)
    # L[i][j] contains the length of the LCS of X[0..i-1] and Y[0..j-1].
    L = [[None]*(n + 1) for i in range(m + 1)]
    sequence = [[None]*(n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
                sequence[i][j] = []
            elif string_list_1[i-1] == string_list_2[j-1]:
                L[i][j] = L[i-1][j-1] + 1
                sequence[i][j] = sequence[i-1][j-1].copy() + [(i-1, j-1)]
                #print("{} and {} are the same".format(i-1, j-1))
                #print("sequence[i-1][j-1]:", sequence[i-1][j-1])
                #print("sequence[i][j]:", sequence[i][j])
            else:
                if L[i-1][j] > L[i][j-1]:
                    L[i][j] = L[i-1][j]
                    sequence[i][j] = sequence[i-1][j]
                else:
                    L[i][j] = L[i][j-1]
                    sequence[i][j] = sequence[i][j-1]
    lcs = sequence[m][n]
    if not lcs:
        return None, None
    str_sequence = []
    skipped = []
    for i_common in range(len(lcs)):
        # Some words are skipped in both strings between the last common and the
        # current common index.
        last_common_index_1st_word = -1 if i_common == 0 else lcs[i_common-1][0]
        last_common_index_2nd_word = -1 if i_common == 0 else lcs[i_common-1][1]
        if (lcs[i_common][0] > last_common_index_1st_word + 1
                and lcs[i_common][1] > last_common_index_2nd_word + 1):
            str_sequence.append('[X/Y]')
            skipped_str_1 = ' '.join(
                string_list_1[last_common_index_1st_word + 1 : lcs[i_common][0]])
            skipped_str_2 = ' '.join(
                string_list_2[last_common_index_2nd_word + 1 : lcs[i_common][1]])
            skipped.append((skipped_str_1, skipped_str_2))
        str_sequence.append(string_list_1[lcs[i_common][0]])
    if (len(string_list_1) > lcs[i_common][0] + 1 and
            len(string_list_2) > lcs[i_common][1] + 1):
        str_sequence.append('[X/Y]')
        skipped_str_1 = ' '.join(string_list_1[lcs[i_common][0] + 1 :])
        skipped_str_2 = ' '.join(string_list_2[lcs[i_common][1] + 1 :])
        skipped.append((skipped_str_1, skipped_str_2))
    return ' '.join(str_sequence), skipped

In [78]:
translated_phrases

['Αζερμπαϊτζάν, που έχει την πρωτεύουσα Μπακού.',
 'Brody Raion, που έχει την πρωτεύουσα Brody.',
 'Cook County, το οποίο έχει την πρωτεύουσα Σικάγο.',
 'Fort Bend County, η οποία έχει την πρωτεύουσα της πόλης Richmond.',
 'Cailuga County, η οποία έχει την πρωτεύουσα Auburn.',
 'Πόλη Hawkesbury, που έχει την πρωτεύουσα του Windsor.',
 'Grand Est, που έχει την πρωτεύουσα του Στρασβούργου.',
 'Caddo Parish, η οποία έχει την πρωτεύουσα Shreveport.',
 'Η Νεμπράσκα, η οποία έχει την πρωτεύουσα Lincoln.',
 'Hampshire, η οποία έχει την πρωτεύουσα της πόλης Winchester.']

In [96]:
final_templates = get_templates_from_populated_translations(
        translated_phrases, en_tuples.values(), translated_tuples.values())
final_templates

searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Το', 'κεφάλαιο', 'του', 'Brody', 'Raion', ',', 'Brody', '.']
searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Cook', "County's", 'Capital', ',', 'Σικάγο', '.']
searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Το', 'κεφάλαιο', 'της', 'κομητείας', 'Fort', 'Bend', ',', 'Richmond', '.']
searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Το', 'κεφάλαιο', 'της', 'Cayuga', 'County', ',', 'Auburn', '.']
searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Πόλη', 'της', 'πρωτεύουσας', 'του', 'Hawkesbury', ',', 'Windsor', '.']
Error in lcs, there aren't two '[X/Y]': [X/Y] του [X/Y] , [X/Y] .
searching lcs between: ['Το', 'κεφάλαιο', 'του', 'Αζερμπαϊτζάν', ',', 'το', 'Μπακού', '.'] ['Κεφάλαιο', 'του', 'Grand', 'Est', ',', 'Στρασβούργο', '

['Το κεφάλαιο του [X] , [Y] .',
 '[X] , [Y] .',
 'Το κεφάλαιο [X] , [Y] .',
 'Το κεφάλαιο του [X] , το [Y] .',
 'Το κεφάλαιο [X] , το [Y] .',
 'Το κεφάλαιο της [X] , [Y] .']

In [75]:
def set_cleaned_words(list, index=None):
    output = set()
    for string in list:
        if index is not None:
            string = string[index]
        for word in string.split(' '):
            word = re.sub('[.,:]', '', re.sub('[.,:]', '', word))
            word = re.sub(' +', ' ', word)
            output.add(word.lower())
    return output

def get_object_subject_order(set_first_position, set_second_position, en_tuples, translated_tuples):
    set_first_position = set_cleaned_words(set_first_position)
    set_second_position = set_cleaned_words(set_second_position)
    subj_en = set_cleaned_words(en_tuples, 0)
    obj_en = set_cleaned_words(en_tuples, 1)
    subj_translated = set_cleaned_words(translated_tuples, 0)
    obj_translated = set_cleaned_words(translated_tuples, 1)
    total_subj = len(subj_en) + len(subj_translated)
    total_obj = len(obj_en) + len(obj_translated)
    first_subj = (len(set_first_position.intersection(subj_en)) + 
                  len(set_first_position.intersection(subj_translated)))/total_subj
    first_obj = (len(set_first_position.intersection(obj_en)) + 
                 len(set_first_position.intersection(obj_translated)))/total_obj
    second_subj = (len(set_second_position.intersection(subj_en)) + 
                   len(set_second_position.intersection(subj_translated)))/total_subj
    second_obj = (len(set_second_position.intersection(obj_en)) + 
                  len(set_second_position.intersection(obj_translated)))/total_obj
    if first_subj > second_subj and first_obj < second_obj:
        return ["[X]", "[Y]"]
    elif first_subj < second_subj and first_obj > second_obj:
        return ["[Y]", "[X]"]
    else:
        print("Error: couldn't conclude on order of X and Y")
        print("first_subj={}, first_obj={}, second_subj={}, second_obj={}".format(
            first_subj, first_obj, second_subj, second_obj))
        print("set_first_position:", set_first_position)
        print("set_second_position:", set_second_position)
        print("subj_en:", subj_en)
        print("subj_translated:", subj_translated)
        print("obj_en:", obj_en)
        print("obj_translated:", obj_translated)
        return None, None

In [95]:
def get_words_list(sentence):
    sentence = sentence.replace(',', ' , ')
    sentence = sentence.replace('.', ' . ')
    sentence = sentence.replace(':', ' : ')
    sentence = re.sub(' +', ' ', sentence)
    sentence = re.sub(' $', '', sentence)
    return sentence.split(' ')


def get_templates_from_populated_translations(translated_phrases: List, en_tuples: List, translated_tuples: List):
    """Returns the templates that are present in more than one translation.

    The template is defined as the longest common subsequence of words between
    more than two translations. It assumes that [X] and [Y] are positioned where
    words are skipped (i.e. do not match) in both of the translations. For
    defining which one is [X] and which one [Y] it simply compares the words
    skipped with the tuples passed as arguments.

    Args:
        translated_phrases: list with the translated phrases.
        en_tuples: List containing the tuples (subject, object) used to populate the templates. 
        en_tuples: List containing the tuples (subject, object) that are the translation of the tuples in english. 
    Returns:
        List of templates where each contains exactly one [X] and one [Y].
    """
    potential_templates = collections.defaultdict(int)
    # Each set contains the phrases for the object/subject found.
    non_overlapping_phrases = [set(), set()]
    for i in range(0, len(translated_phrases)):
        for j in range(i+1, len(translated_phrases)):
            lcs, non_overlapping = longest_common_subsequence(
                get_words_list(translated_phrases[i]),
                get_words_list(translated_phrases[j]))
            if not lcs:
                continue
            if lcs.count("[X/Y]") != 2:
                print("Error in lcs, there aren't two '[X/Y]':", lcs)
                continue
            for subject_or_object in non_overlapping[0]:
                non_overlapping_phrases[0].add(subject_or_object)
            for subject_or_object in non_overlapping[1]:
                non_overlapping_phrases[1].add(subject_or_object)
            potential_templates[lcs] += 1
    print("potential_templates:", potential_templates)
    first_str, second_str = get_object_subject_order(
        *non_overlapping_phrases, en_tuples, translated_tuples)
    if first_str is None:
        return []
    final_templates = []
    for template, _ in filter(lambda x: x[1] > 1, potential_templates.items()):
        final_tempalte = template.replace("[X/Y]", first_str, 1)
        final_templates.append(final_tempalte.replace("[X/Y]", second_str, 1))
    return final_templates

In [81]:
lang = "el"

for template in templates[4:5]:
    print(">>>>>>> template:", template["pattern"])
    en_tuples = get_k_subject_object_tuples(
        os.path.join(tuples_folder, "en", "triples", relation_filename),
        K_POPULATED_TEMPLATES)
    print("en_tuples:\n", en_tuples.items())
    translated_tuples = get_subject_object_tuples_from_lines(
        os.path.join(tuples_folder, lang, "triples", relation_filename),
        en_tuples.keys())
    print("translated_tuples:\n", translated_tuples.items())
    populated_phrases = get_populated_phrases(template['pattern'], en_tuples.values())
    print("populated_phrases:\n", populated_phrases)
    translated_phrases = [translator.translate(text, "en", lang)
                          for text in populated_phrases]
    print("translated_phrases:\n", translated_phrases)
    final_templates = get_templates_from_populated_translations(
        translated_phrases, en_tuples.values(), translated_tuples.values())
    print("final_templates:", final_templates)
    print()

>>>>>>> template: [X]'s capital, [Y].
en_tuples:
 dict_items([(0, ('Azerbaijan', 'Baku')), (1, ('Brody Raion', 'Brody')), (2, ('Cook County', 'Chicago')), (3, ('Fort Bend County', 'Richmond')), (4, ('Cayuga County', 'Auburn')), (5, ('City of Hawkesbury', 'Windsor')), (6, ('Grand Est', 'Strasbourg')), (7, ('Caddo Parish', 'Shreveport')), (8, ('Nebraska', 'Lincoln')), (9, ('Hampshire', 'Winchester'))])
translated_tuples:
 dict_items([(0, ('Αζερμπαϊτζάν', 'Μπακού')), (2, ('Κομητεία Κουκ', 'Σικάγο')), (6, ('Γκραντ Εστ', 'Στρασβούργο')), (8, ('Νεμπράσκα', 'Λίνκολν')), (9, ('Χάμπσαϊρ', 'Γουίντσεστερ'))])
populated_phrases:
 ["Azerbaijan's capital, Baku.", "Brody Raion's capital, Brody.", "Cook County's capital, Chicago.", "Fort Bend County's capital, Richmond.", "Cayuga County's capital, Auburn.", "City of Hawkesbury's capital, Windsor.", "Grand Est's capital, Strasbourg.", "Caddo Parish's capital, Shreveport.", "Nebraska's capital, Lincoln.", "Hampshire's capital, Winchester."]
translated_p